In [7]:
import pandas as pd
import numpy as np


In [8]:
def calculate_distance(df, name1, name2, x_col, y_col, z_col, print_values=True):
    point1 = df[df["Mic_Index"] == name1][[x_col, y_col, z_col]].iloc[0]
    point2 = df[df["Mic_Index"] == name2][[x_col, y_col, z_col]].iloc[0]
    
    # euklidische Distanz berechnen
    dist = np.sqrt((point2[x_col] - point1[x_col])**2 + 
                   (point2[y_col] - point1[y_col])**2 + 
                   (point2[z_col] - point1[z_col])**2)
    
    if print_values == True:
        print(name1)
        print(x_col, ":", round(point1[x_col], 3))
        print(y_col, ":", round(point1[y_col], 3))
        print(z_col, ":", round(point1[z_col], 3))
        print("")
        print(name2)
        print(x_col, ":", round(point2[x_col], 3))
        print(y_col, ":", round(point2[y_col], 3))
        print(z_col, ":", round(point2[z_col], 3))

        print("distance = ", round(dist, 3))


    return dist

In [9]:

file_path = r"mic_positions\data\side_postions_measured.xlsx"
df = pd.read_excel(file_path, engine='openpyxl')

# cut empty rows
df = df.head(64)

In [ ]:
df_3d = df.copy()


# all distances in meteres
# grid = innenseite vom gitter an den Seiten
# frame = unterseite vom Aluprofil 

MicOffsetInside = 0.028 # wie weit die mics vom grid nach innen reichen
MicOffsetInsideTop = 0.013 # wie weit die mics vom frame nach innen reichen

side_grid_width = 1.378 #measured width of side grid
top_grid_width = 1.23 # measured width of top frame ( 2D-Y direction )

TopFrameYOffset = 0.069 # Y distance between front plane and edge of top grid (Gitter zu Aluprofilkante)

# X distance between edge of top frame and left / right grid
XSideOffsetLeft = 0.01
XSideOffsetRight = 0.01

# Y  distance between front grid and edge of left/right grid
YSideOffsetLeft = 0.088
YSideOffsetRight = 0.085

# distance between top frame and lower edge of side grids (Z = 0)
TopOffset = 1.884

x_2d = "X measured"
y_2d = "Y measured"

# translate 2d coordinates to 3d coordinates
# for left plane
df_3d.loc[df_3d["Plane"] == "left", "X"] = 0 + MicOffsetInside
df_3d.loc[df_3d["Plane"] == "left", "Y"] = df[x_2d] + YSideOffsetLeft
df_3d.loc[df_3d["Plane"] == "left", "Z"] = df[y_2d]

# for front plane
df_3d.loc[df_3d["Plane"] == "front", "X"] = -df[x_2d]+side_grid_width  + XSideOffsetLeft
df_3d.loc[df_3d["Plane"] == "front", "Y"] = 0 + MicOffsetInside
df_3d.loc[df_3d["Plane"] == "front", "Z"] = df[y_2d]

# for right plane
df_3d.loc[df_3d["Plane"] == "right", "X"] = side_grid_width + XSideOffsetLeft + XSideOffsetRight - MicOffsetInside
df_3d.loc[df_3d["Plane"] == "right", "Y"] = -df[x_2d]+side_grid_width  + YSideOffsetRight
df_3d.loc[df_3d["Plane"] == "right", "Z"] = df[y_2d]

# for top plane
df_3d.loc[df_3d["Plane"] == "top", "X"] = df[x_2d] + XSideOffsetLeft
df_3d.loc[df_3d["Plane"] == "top", "Y"] = -df[y_2d] + top_grid_width + TopFrameYOffset
df_3d.loc[df_3d["Plane"] == "top", "Z"] = TopOffset - MicOffsetInsideTop


columns_order = [
    "Unnamed: 0",
    "Mic_Index",
    "Plane",
    "X",
    "Y",
    "Z",
    "X measured",
    "Y measured",
]
df_3d = df_3d[columns_order]


# Ergebnis anzeigen
print(df_3d)


    Unnamed: 0 Mic_Index  Plane      X      Y      Z  X measured  Y measured
0          0.0        B1  right  1.370  0.814  0.020       0.649       0.020
1          1.0        B2  right  1.370  0.561  0.266       0.902       0.266
2          2.0        B3  right  1.370  1.212  0.287       0.251       0.287
3          3.0        B4  right  1.370  0.150  0.380       1.313       0.380
4          4.0        B5  right  1.370  0.901  0.425       0.562       0.425
..         ...       ...    ...    ...    ...    ...         ...         ...
59        59.0        A9    top  0.075  0.521  1.858       0.065       0.778
60        60.0       A10    top  0.800  0.393  1.858       0.790       0.906
61        61.0       A11    top  0.495  0.155  1.858       0.485       1.144
62        62.0       A12    top  1.148  0.129  1.858       1.138       1.170
63        63.0       A13    top  0.208  0.117  1.858       0.198       1.182

[64 rows x 8 columns]


In [11]:
_ = calculate_distance(df_3d, "D9", "B6", x_col="X", y_col="Y", z_col="Z")

D9
X : 0.028
Y : 0.629
Z : 0.773

B6
X : 1.37
Y : 0.559
Z : 0.629
distance =  1.352


In [12]:
# fancy interactiver plot 
# damit kann man die namen der mikrofone sehr gut sehen


import plotly.express as px

fig = px.scatter_3d(df_3d, x='X', y='Y', z='Z', text='Mic_Index', title='3D Scatter Plot')

# Plot anzeigen
fig.show()